# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify
import numpy as np
from macrohet import visualise
import os
import re
import numpy as np
import cv2
import btrack
import zarr
from skimage import io
import cv2
from tqdm.auto import tqdm

def trim_black_borders(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply binary thresholding to get black areas
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = max(contours, key=cv2.contourArea)  # find the largest contour
        x, y, w, h = cv2.boundingRect(cnt)
        image = image[y:y+h, x:x+w]
    return image

def crop_image_to_tiles(image, num_tiles_per_row, overlap_percentage):
    height, width, _ = image.shape

    # Calculate basic tile dimensions
    basic_tile_width = width // num_tiles_per_row
    basic_tile_height = height // num_tiles_per_row

    # Calculate the overlap in pixels
    overlap = int(overlap_percentage * basic_tile_width)

    # Calculate effective tile dimensions including overlap
    tile_width = basic_tile_width + overlap
    tile_height = basic_tile_height + overlap

    tiles = []
    for i in range(num_tiles_per_row):
        for j in range(num_tiles_per_row):
            # Calculate the position of the tile accounting for overlap
            left = max(0, i * basic_tile_width - overlap // 2)
            upper = max(0, j * basic_tile_height - overlap // 2)
            right = min(width, left + tile_width)
            lower = min(height, upper + tile_height)
            
            # Crop the tile
            crop = image[upper:lower, left:right]
            tiles.append(crop)

    return tiles

num_tiles_per_row = 3
overlap_percentage =0.1

tile_position_mapping_dictionary = {1:2, 
                                   2:7, 
                                   3:8, 
                                   4:3, 
                                   5:6, 
                                   6:9, 
                                   7:4, 
                                   8:5,
                                   9:1}

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'ND0003'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 32.9 s, sys: 2.64 s, total: 35.6 s
Wall time: 35.8 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,0,0.135256499,0,2024-02-16T17:15:25.597+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
1,0301K1F1P1R2,Ok,r03c01f01p01-ch2sk1fk1fl1.tiff,3,1,1,1,0,2,1,...,0,0.135256499,0,2024-02-16T17:15:25.813+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
2,0301K1F1P2R1,Ok,r03c01f01p02-ch1sk1fk1fl1.tiff,3,1,1,2,0,1,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.157+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
3,0301K1F1P2R2,Ok,r03c01f01p02-ch2sk1fk1fl1.tiff,3,1,1,2,0,2,1,...,2E-06,0.1352586,0,2024-02-16T17:15:26.39+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
4,0301K1F1P3R1,Ok,r03c01f01p03-ch1sk1fk1fl1.tiff,3,1,1,3,0,1,1,...,4E-06,0.135260597,0,2024-02-16T17:15:26.733+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349267,0612K154F9P1R2,Ok,r06c12f09p01-ch2sk154fk1fl1.tiff,6,12,9,1,153,2,1,...,0,0.135008901,275402.773,2024-02-19T21:59:46.84+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349268,0612K154F9P2R1,Ok,r06c12f09p02-ch1sk154fk1fl1.tiff,6,12,9,2,153,1,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.183+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349269,0612K154F9P2R2,Ok,r06c12f09p02-ch2sk154fk1fl1.tiff,6,12,9,2,153,2,1,...,2E-06,0.135010898,275402.773,2024-02-19T21:59:47.4+00:00,488,522,40,1.1,0.1,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."
349270,0612K154F9P3R1,Ok,r06c12f09p03-ch1sk154fk1fl1.tiff,6,12,9,3,153,1,1,...,4E-06,0.135012895,275402.773,2024-02-19T21:59:47.743+00:00,640,706,40,1.1,0.2,"[[0.999464,0,0,-5.0],[0,-0.999464,0,4.1],[0,0,..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  Replicate #
Row Column                                                           
3   1         UNI     CTRL             0             EC0            1
    2         UNI     CTRL             0             EC0            2
    3          WT     CTRL             0             EC0            1
    4          WT     CTRL             0             EC0            2
    5          WT      PZA            60            EC50            1
    6          WT      PZA            60            EC50            2
    7          WT      RIF           0.1            EC50            1
    8          WT      RIF           0.1            EC50            2
    9          WT      INH          0.04            EC50            1
    10         WT      INH          0.04            EC50            2
    11         WT      BDQ          0.02            EC50            1
    12         WT      BDQ          0.02            EC50            2
4   3          WT     CTRL             0             EC0            3
    4          WT     CTRL             0             EC0            4
    5          WT      PZA           400            EC99            1
    6          WT      PZA           400            EC99            2
    7          WT      RIF             2            EC99            1
    8          WT      RIF             2            EC99            2
    9          WT      INH             2            EC99            1
    10         WT      INH             2            EC99            2
    11         WT      BDQ           2.5            EC99            1
    12         WT      BDQ           2.5            EC99            2
5   3         RD1     CTRL             0             EC0            1
    4         RD1     CTRL             0             EC0            2
    5         RD1      PZA            60            EC50            1
    6         RD1      PZA            60            EC50            2
    7         RD1      RIF           0.1            EC50            1
    8         RD1      RIF           0.1            EC50            2
    9         RD1      INH          0.04            EC50            1
    10        RD1      INH          0.04            EC50            2
    11        RD1      BDQ          0.02            EC50            1
    12        RD1      BDQ          0.02            EC50            2
6   3         RD1     CTRL             0             EC0            3
    4         RD1     CTRL             0             EC0            4
    5         RD1      PZA           400            EC99            1
    6         RD1      PZA           400            EC99            2
    7         RD1      RIF             2            EC99            1
    8         RD1      RIF             2            EC99            2
    9         RD1      INH             2            EC99            1
    10        RD1      INH             2            EC99            2
    11        RD1      BDQ           2.5            EC99            1
    12        RD1      BDQ           2.5            EC99            2

### Iterate and save out 

In [6]:
for acq_ID, info in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    # if acq_ID[0] != 6:
    #     continue
    # if acq_ID == (6, 3):
    #     continue
    output_basedir = os.path.join(base_dir, F'postfix/labelled_tracks_final_frame/{acq_ID}')
    if os.path.exists(output_basedir): 
        continue    
    try:
        os.makedirs(output_basedir, exist_ok=True)  # Make sure the base directory exists
    
        cropped_metadata = metadata[(metadata['Row'] == str(acq_ID[0]))
                                    & (metadata['Col'] == str(acq_ID[1]))
                                    & (metadata['TimepointID'] == str(metadata['TimepointID'].astype(int).max()))
                                    & (metadata['ChannelID'] == '2')
                                    & (metadata['PlaneID'] == '1')
                                    ]
        
        
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_group = zarr.open(image_dir, mode='r')
        
        images = zarr_group.images[-1,:,0,...]
        images.shape
        
        viewer = napari.Viewer(title = f'{expt_ID, acq_ID}')
        
        viewer.add_image(images,
                         channel_axis = 0, 
                         # # scale = napari_scale, 
                         contrast_limits=[[280, 1000],[0,3000]])
        
        with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/{expt_ID}/labels/cpv3/{acq_ID}.h5'), 
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                        segmentation = reader.segmentation
                        tracks = reader.tracks
        
        last_frame_tracks = [t for t in tracks if t.in_frame(149) and len(t) >= 70]
        coords = [(int(t.y[-1]*5.04), int(t.x[-1]*5.04)) for t in last_frame_tracks]
        # Assuming last_frame_tracks is a list of objects with attributes ID, x, and y
        id_xy_dict = [{'ID':t.ID, 
                      'x':int(t.x[-1]*5.04),
                      'y':int(t.y[-1]*5.04)} for t in last_frame_tracks]
        
        ### JUST IDs
        text_parameters = {
            'string': '{ID}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs.png'
        output_path = os.path.join(output_basedir, 'tiled', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs and COORDS
        text_parameters = {
            'string': '{ID}\n{x},{y}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_xy.png'
        output_path = os.path.join(output_basedir, 'tiled/with_xy_coords', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs with MASKS
        text_parameters = {
            'string': '{ID}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        viewer.add_labels(segmentation[-1])
        viewer.layers['Labels'].contour = 10
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_masks.png'
        output_path = os.path.join(output_basedir, 'tiled/with_masks', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        
        ### IDs and COORDS and masks
        text_parameters = {
            'string': '{ID}\n{x},{y}',
            'size': 9,
            'color': 'white',
            'anchor': 'center',
            # 'translation': [-3, 0],
        }
        viewer.add_points(coords, size = 150, face_color='transparent', edge_color='transparent', edge_width=0.1, 
                          properties = id_xy_dict,
                          text = text_parameters)
        # viewer.add_labels(segmentation[-1])
        # viewer.layers['Labels'].contour = 10
        output_fn = f'{acq_ID}_{expt_ID}_t-1_IDs_xy_masks.png'
        output_path = os.path.join(output_basedir, 'tiled/with_xy_coords_masks', output_fn)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Make sure the directory exists
        screen_shot = trim_black_borders(viewer.screenshot())
        io.imsave(output_path, screen_shot)
        tiles = crop_image_to_tiles(screen_shot, 3, 0.1)
        for i, tile in enumerate(tiles): 
            FieldID = tile_position_mapping_dictionary[i+1]
            tile_fn = cropped_metadata[cropped_metadata['FieldID'] == str(FieldID)]['URL'].iloc[0]
            directory, old_filename = os.path.split(output_path)
            new_filename = f"{tile_fn}_{old_filename}"
            new_directory = directory.replace('tiled', 'untiled')    
            new_path = os.path.join(new_directory, new_filename)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)  # Make sure the directory exists
            io.imsave(new_path, tile)
        del viewer.layers['Points']
        del viewer.layers['Labels']
        viewer.close()
    except:
        print(acq_ID)

  0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2024/04/24 09:58:28 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 6).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 6).h5...


(5, 6)


[INFO][2024/04/24 09:58:32 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 7).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 7).h5...


(5, 7)


[INFO][2024/04/24 09:58:36 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 8).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 8).h5...


(5, 8)


[INFO][2024/04/24 09:58:42 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 9).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 9).h5...
[INFO][2024/04/24 10:00:11 am] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/04/24 10:00:12 am] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/04/24 10:00:12 am] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/04/24 10:00:12 am] Loading objects/obj_type_1 (40247, 5) (40247 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (40247, 5) (40247 filtered: None)
[INFO][2024/04/24 10:00:14 am] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 9).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 9).h5
[INFO][2024/04/24 10:00:50 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(5, 10).h5...
INFO